In [1]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy
from datas import *

# 一、数据准备

## 1.1 读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

In [2]:
year = 2018
end_year = 2021
list_assets, df_assets = get_hs300_stocks(f'{year}-01-01')
# df_org= get_all_date_data(f'{year}-01-01', f'{year+1}-01-01', list_assets)
df_org= get_all_date_data(f'{year}-01-01', f'{end_year + 1}-01-01', list_assets)
df1 = df_org.rename(columns={
        "date": "datetime", 
        "asset": "sec_code"})
df1["openinterest"] = 0
daily_price=df1[['sec_code','datetime', "open", "close", "high", "low", "volume", 'openinterest']]
daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])

# 以 datetime 为 index，类型为 datetime 或 date 类型，Datafeeds 默认情况下是将 index 匹配给 datetime 字段；
daily_price = daily_price.set_index(['datetime'])
daily_price

login success!
login respond error_code:0
login respond  error_msg:success
query_hs300 error_code:0
query_hs300  error_msg:success
logout success!
300


/var/folders/mj/q763xcj90k17wtcw8c2h9n_c0000gn/T/ipykernel_82864/2478071887.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])


,sec_code,open,close,high,low,volume,openinterest
datetime,,,,,,,
2018-01-02,600000,12.61,12.72,12.77,12.60,313231,0
2018-01-03,600000,12.73,12.66,12.80,12.66,378391,0
2018-01-04,600000,12.70,12.66,12.73,12.62,278838,0
2018-01-05,600000,12.67,12.69,12.71,12.62,310267,0
2018-01-08,600000,12.69,12.68,12.71,12.63,313899,0
...,...,...,...,...,...,...,...
2021-12-27,300315,4.63,4.47,4.63,4.44,972028,0
2021-12-28,300315,4.48,4.47,4.54,4.44,744388,0
2021-12-29,300315,4.45,4.45,4.51,4.40,718864,0


## 1.2 生成调仓信息表

表内数据说明：

+ trade_date： 调仓期（每月最后一个交易日）;

+ sec_code：持仓成分股；

+ weight：持仓权重。

In [3]:
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

df_2 = df_org[['date', 'asset', "close"]]
df_2['date'] = pd.to_datetime(df_2['date'])
# print(df_all)

close = df_2.pivot(index='date', columns='asset', values='close')

alpha_num = 19
alpha_name = 'Myalphas'

# 读取已经计算好的因子
# alpha = pd.read_csv('alphas/{}/{}/alpha{:03d}.csv'.format(alpha_name, year, alpha_num))
alpha = pd.read_csv('alphas/{}/Total/alpha{:03d}.csv'.format(alpha_name, alpha_num))

# 筛选出今年的数据，需与股票收盘日期区间一致
alpha = alpha[(alpha['date'] >= f'{year}-01-01') & (alpha['date'] <= f'{end_year + 1}-01-01')]

# 因子矩阵转换为一维数据(alphalens需要的格式)
alpha = alpha.melt(id_vars=['date'], var_name='asset', value_name='factor' )

# date列转为日期格式
alpha['date'] = pd.to_datetime(alpha['date'])
alpha = alpha[['date', 'asset', 'factor']]

# 设置二级索引
alpha = alpha.set_index(['date', 'asset'], drop=True)
alpha.sort_index(inplace=True)


ret = get_clean_factor_and_forward_returns(alpha, close,quantiles=5)
ret = ret.reset_index()
ret = ret[ret['factor_quantile'] == 5]
# ret['week'] =  pd.to_datetime(ret['date']).dt.weekday
# ret = ret[ret['week'] == 4]
ret = ret[['date','asset']]
ret['weight'] = 1/60
trade_info = ret.rename(columns={
        "date": "trade_date", 
        "asset": "sec_code"})
trade_info

/var/folders/mj/q763xcj90k17wtcw8c2h9n_c0000gn/T/ipykernel_82864/3780514021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['date'] = pd.to_datetime(df_2['date'])


ValueError: cannot handle a non-unique multi-index!

# 二、 回测分析

## 2.1 加载策略和数据

In [ ]:
# 回测策略
class TestStrategy(bt.Strategy):
    params = (
        ('buy_stocks', None), # 传入各个调仓日的股票列表和相应的权重
    )
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def __init__(self):
         
        self.trade_dates = pd.to_datetime(self.p.buy_stocks['trade_date'].unique()).tolist()
        self.buy_stock = self.p.buy_stocks # 保留调仓信息
        self.order_list = []  # 记录以往订单，在调仓日要全部取消未成交的订单
        self.buy_stocks_pre = [] # 记录上一期持仓
    
    def next(self):
        # 获取当前的回测时间点
        dt = self.datas[0].datetime.date(0)
        # 打印当前时刻的总资产
        self.log('当前总资产 %.2f' %(self.broker.getvalue()))
        
        # 如果是调仓日，则进行调仓操作
        if pd.to_datetime(dt) in self.trade_dates:
            print(f"--------------{dt} 为调仓日----------")
            #取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                print("--------------- 撤销未完成的订单 -----------------")
                for od in self.order_list:
                    # 如果订单未完成，则撤销订单
                    self.cancel(od) 
                 #重置订单列表
                self.order_list = [] 

            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list)  # 打印持仓列表

            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock)
            if sell_stock:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data)  
                        self.order_list.append(od) # 记录卖出订单

            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)

            self.buy_stocks_pre = long_list  # 保存此次调仓的股票列表
        
    #订单日志    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm,
                     order.executed.size,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref,
                         order.executed.price,
                         order.executed.value,
                         order.executed.comm,
                         order.executed.size,
                         order.data._name))

# 实例化大脑
cerebro_ = bt.Cerebro() 

# 按股票代码，依次循环传入数据
for stock in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique())
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    # data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime.datetime(year,1,1), todate=datetime.datetime(end_year + 1,1,1))
    cerebro_.adddata(datafeed, name=stock)
    print(f"{stock} Done !") 

600000 Done !
600008 Done !
600009 Done !
600010 Done !
600011 Done !
600015 Done !
600016 Done !
600018 Done !
600019 Done !
600021 Done !
600023 Done !
600028 Done !
600029 Done !
600030 Done !
600031 Done !
600036 Done !
600038 Done !
600048 Done !
600050 Done !
600061 Done !
600066 Done !
600068 Done !
600074 Done !
600085 Done !
600089 Done !
600100 Done !
600104 Done !
600109 Done !
600111 Done !
600115 Done !
600118 Done !
600153 Done !
600157 Done !
600170 Done !
600177 Done !
600188 Done !
600196 Done !
600208 Done !
600219 Done !
600221 Done !
600233 Done !
600271 Done !
600276 Done !
600297 Done !
600309 Done !
600332 Done !
600340 Done !
600352 Done !
600362 Done !
600369 Done !
600372 Done !
600373 Done !
600376 Done !
600383 Done !
600390 Done !
600406 Done !
600415 Done !
600436 Done !
600482 Done !
600485 Done !
600489 Done !
600498 Done !
600518 Done !
600519 Done !
600522 Done !
600535 Done !
600547 Done !
600549 Done !
600570 Done !
600583 Done !
600585 Done !
600588

## 2.2 添加分析器并执行回测

In [ ]:
cerebro = deepcopy(cerebro_)  # 深度复制已经导入数据的 cerebro_，避免重复导入数据 
# 初始资金 100,000,000    
cerebro.broker.setcash(100000.0) 
# cerebro.broker.setcommission(commission=0.0015)
# 添加策略
cerebro.addstrategy(TestStrategy, buy_stocks=trade_info) # 通过修改参数 buy_stocks ，使用同一策略回测不同的持仓列表

# 添加分析指标
# 返回年初至年末的年度收益率
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
# 计算最大回撤相关指标
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
# 计算年化收益
cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns', tann=252)
# 计算年化夏普比率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0) # 计算夏普比率
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A')
# 返回收益率时序
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')

# 启动回测
result = cerebro.run()

## 2.3 回测结果分析

In [ ]:
strat = result[0]
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())
print("--------------- Returns -----------------")
print(strat.analyzers._Returns.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- SharpeRatio_A -----------------")
print(strat.analyzers._SharpeRatio_A.get_analysis())

# 绘制累计收益率
# ret = pd.Series(strat.analyzers._TimeReturn.get_analysis())
# (ret + 1).cumprod().plot(figsize=(12,6))

# ret = [{
#         '年度': year,
#         '收益率': strat.analyzers._Returns.get_analysis()['rtot'],
#         '日均收益率': strat.analyzers._Returns.get_analysis()['ravg'],
#         '年化收益率': strat.analyzers._Returns.get_analysis()['rnorm'],
#         '最大回撤(%)': strat.analyzers._DrawDown.get_analysis()['max']['drawdown']
#         * (-1),
#         '夏普比率': strat.analyzers._SharpeRatio.get_analysis()['sharperatio'],
#     }]
# pd.DataFrame(ret).sort_values(by='年化收益率', ascending=False)

--------------- AnnualReturn -----------------
OrderedDict([(2018, 34.28250910000015), (2019, -1.4889438432824031), (2020, 0.1487993715624727)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 486), ('drawdown', 149.8192332454337), ('moneydown', 5959820.290000041), ('max', AutoOrderedDict([('len', 486), ('drawdown', 198.95385616257084), ('moneydown', 7914399.260000076)]))])
--------------- Returns -----------------
OrderedDict([('rtot', -inf), ('ravg', -inf), ('rnorm', -inf), ('rnorm100', -inf)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', 0.32645861297702833)])
--------------- SharpeRatio_A -----------------
OrderedDict([('sharperatio', 0.6652847481750662)])
